In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import json
import baseline
from baseline import *
from decoder import *
from alphabet import *
from train import *
import torch
from matplotlib import pyplot as plt

torch.__version__

'1.4.0'

In [2]:
LANGUAGE = 'Python'

def filter_ascii(strings):
    'Returns only the strings that can be encoded in ASCII.'
    l = []
    for s in strings:
        try:
            s.encode('ascii')
            if len(s) <= 80:
                l.append(s)
        except UnicodeEncodeError:
            pass
        
    return l

with open('dataset/medium.json') as f:
    multilang_dataset = json.load(f)
    dataset = multilang_dataset[LANGUAGE]
    
    dataset['train'] = filter_ascii(dataset['train'])
    dataset['dev'] = filter_ascii(dataset['dev'])
    dataset['test'] = filter_ascii(dataset['test'])
    
    tiny_dataset = {
        'train': dataset['train'][:50],
        'dev': dataset['train'][:50],
        'test': dataset['train'][:50],
    }
    
    print('{} training examples, {} validation examples, {} test exampless'.format(
        len(dataset['train']), 
        len(dataset['dev']),
        len(dataset['test'])))

76738 training examples, 9590 validation examples, 9616 test exampless


In [3]:
def generate_dumb_dataset():
    'Returns a dataset where all examples are the same string, which consists of 10 times the same letter.'

    SIZE = 200
    l = []

    for i in range(SIZE):
        l.append(random.choice('abcdefghijklmnopqrstuvwxyz') * random.choice([5, 10]))
        
    return {'train': l, 'dev': l, 'test': l}

dumb_dataset = generate_dumb_dataset()

In [67]:
device = torch.device(0) if torch.cuda.is_available() else torch.device('cpu')
print(device)

alphabet = AsciiOneHotEncoding(device)
encoder = baseline.UniformEncoder(0.9)
decoder = AutoCompleteDecoderModel(alphabet, hidden_size=64)

cpu


In [ ]:
parameters = {
    'learning_rate': 5e-3,
    'verbose': True,
    'batch_size': 32,
    'init_scale': 0.01,
    'epochs': 200,
}

train_loss_history = train(encoder, decoder, dumb_dataset, parameters, device)
plt.plot(train_loss_history)

Epoch 0 iteration 0: loss = 4.852, tp = 701.94 lines/s, ETA 00h01m03s
Epoch 14 iteration 2: loss = 3.294, tp = 872.37 lines/s, ETA 00h00m47s
Epoch 28 iteration 4: loss = 3.143, tp = 911.52 lines/s, ETA 00h00m42s
Epoch 42 iteration 6: loss = 3.168, tp = 920.22 lines/s, ETA 00h00m38s
Epoch 57 iteration 1: loss = 3.194, tp = 926.27 lines/s, ETA 00h00m34s
Epoch 71 iteration 3: loss = 3.086, tp = 931.01 lines/s, ETA 00h00m30s


# End-to-end samples

In [97]:
SPLIT = 'train'

import copy

s = random.choice(dumb_dataset[SPLIT])
compressed = encoder.encode(s)
decompressed = decoder([compressed])

print('String:', repr(s))
print('Encoded:', repr(compressed))
print('Decoded:', repr(decompressed[0]))

String: 'llllllllll'
Encoded: 'llllllll'
Decoded: 'llllllll'
